### Research 

In [ ]:
# checking cities btw the two df (uscities and modified fandom)

df = pd.read_csv(r'C:\Users\inesn\Documents\Mines\2A\TR\econometrics\data_collection\plein_de_data\fandom_traitées.csv', parse_dates = ["Opening_date", "Closing_date"])

n_tot = 0
for state in df.State.unique():
    for city in df[df.State == state].Town.unique():
        if city not in uscities[uscities.state_id == state].city.unique() :
            n_tot+=1
print(n_tot)

In [ ]:
import geopandas as gpd
import contextily as cx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as pa
from matplotlib.lines import Line2D

long_min = - 128
long_max = -65
lat_min = 23
lat_max = 51

#collecte de la dataframe...
df = pd.read_csv('test_data.csv', parse_dates = ["Opening_date", "Closing_date"])
df = df[['Latitude', 'Longitude', 'State']]
df.head()

#conversion en geodataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='EPSG:4326')

#création des couleurs/tailles de points
def couleur(a): # couleur d'un point du graphique en fonction de a la date du séisme en km
    if a == "opened":
        return 'purple'
    elif a == "closed":
        return 'blue'
    elif a == "opened-closed":
        return 'green'
    return 'yellow'

vcouleur = np.vectorize(couleur)

color=vcouleur(df['State'])

fig, ax = plt.subplots() #gdf.plot(kind='scatter', y='Latitude', x='Longitude', s=magn_2, c=prof, edgecolors='k', linewidth = 0.6)
ax.set_xlim(long_min, long_max)
ax.set_ylim(lat_min, lat_max)
cx.add_basemap(ax, crs = gdf.crs, source=cx.providers.OpenStreetMap.Mapnik) #OpenStreetMap.Mapnik

line1 = Line2D(range(1), range(1), color="purple", marker='o', markerfacecolor="purple")
line2 = Line2D(range(1), range(1), color="blue", marker='o',markerfacecolor="blue")
line3 = Line2D(range(1), range(1), color="cyan", marker='o', markerfacecolor="cyan")
line4 = Line2D(range(1), range(1), color="green", marker='o',markerfacecolor="green")
line5 = Line2D(range(1), range(1), color="yellow", marker='o',markerfacecolor="yellow")
#plt.legend((line1,line2,line3,line4, line5),('< 2 km', '2-4 km', '4-6 km', '6-8 km', '8-10 km'),numpoints=1, loc='lower right',title = 'Profondeur',borderpad = 0.5, labelspacing = 0.5)

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as pa
from matplotlib.lines import Line2D

long_min = - 128
long_max = -65
lat_min = 23
long_max = 51

#collecte de la dataframe...
df = pd.read_csv('test_data.csv', parse_dates = ["Opening_date", "Closing_date"])

df.head()

In [ ]:
import geopandas as gpd
import contextily as cx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as pa
from matplotlib.lines import Line2D

long_min = - 128
long_max = -65
lat_min = 23
lat_max = 51

#collecte de la dataframe...
df = pd.read_csv('test_data.csv', parse_dates = ["Opening_date", "Closing_date"])
df = df[['Latitude', 'Longitude', 'State']]
df.head()

#conversion en geodataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='EPSG:4326')

fig, ax = plt.subplots() #gdf.plot(kind='scatter', y='Latitude', x='Longitude', s=magn_2, c=prof, edgecolors='k', linewidth = 0.6)
ax.set_xlim(long_min, long_max)
ax.set_ylim(lat_min, lat_max)
cx.add_basemap(ax, crs = gdf.crs, source=cx.providers.OpenStreetMap.Mapnik) #OpenStreetMap.Mapnik

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

#uscities df
uscities = pd.read_csv('uscities.csv')
uscities = uscities[["city", "state_id", "lat", "lng"]]

#our df
df = pd.read_csv(r'C:\Users\inesn\Documents\Mines\2A\TR\econometrics\data_collection\plein_de_data\fandom_traitées.csv', parse_dates = ["Opening_date", "Closing_date"])

#creating boolean columns for entry/exit in 2016 (no entry and exit in 2016 for a single store)
df['opened_2016'] = (df.Opening_date>='2016-01-31')&(df.Opening_date<'2017-01-31')
df['closed_2016'] = (df.Closing_date>='2016-01-31')&(df.Closing_date<'2017-01-31')

#keeping only stores with entry/exit in 2016
df = df[df.opened_2016 | df.closed_2016]

#designing the red-orange-green color code and the size code
#assuming that there isn't 2 cities with same name in same state in our database
df_test = df.groupby(['Town','State'])['Store_id'].count().rename('count').to_frame()
df2 = df.groupby(['Town','State'])['opened_2016'].any().rename('opened_2016_city').to_frame()
df3 = df.groupby(['Town','State'])['closed_2016'].any().rename('closed_2016_city').to_frame()
df4 = df2.merge(df3, how = "outer", on = ['Town','State'])
df4 = df4.merge(df_test, how = "outer", on = ['Town','State'])
def legend(x):
    if x == 1 :
        return 'Entry'
    if x == 2 :
        return 'Exit'
    return 'Both'
legend_v = np.vectorize(legend)
df4['Walmart entry or/and exit (2016, per city)']= legend_v(1*df4['opened_2016_city']+2*df4['closed_2016_city'])
map_data = pd.merge(df4, uscities,  how='inner', left_on=['Town','State'], right_on = ['city','state_id'])

#plotting the map
geo_map_data = gpd.GeoDataFrame(
    map_data, geometry=gpd.points_from_xy(map_data.lng, map_data.lat))
geo_map_data.crs = 'EPSG:4326'
geo_map_data.explore(column='Walmart entry or/and exit (2016, per city)', cmap=["blue","green","red"], 
                     style_kwds={"style_function": lambda x: {"radius": x["properties"]["count"]*2}})

### Plot Maps (County Maps)

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#counties basemap
shape_path = 'cb_2018_us_county_5m.shp'
shape = gpd.read_file(shape_path)

#withdrawal of counties outside of continental America
mask = (shape.STATEFP=="02")|(shape.STATEFP=="15")|(shape.STATEFP=="60")|(shape.STATEFP=="69")|(shape.STATEFP=="66")|(shape.STATEFP=="78")|(shape.STATEFP=="72")
continent_shape = shape.drop(shape[mask].index)

In [2]:
continent_shape = continent_shape.dropna()

In [3]:
#dots' colors

from matplotlib.colors import LinearSegmentedColormap

def legend2(x):
    if x == 'Entry only' :
        return 0.6
    if x == 'Exit only' :
        return 1
    return 0

legend2_v = np.vectorize(legend2)
geo_map_data['color']= legend2_v(geo_map_data['Walmart store entry or/and exit (2016, per city)'])
cmap = LinearSegmentedColormap.from_list(
    'mycmap', [(0, 'blue'), (0.6, 'green'), (1, 'red')])


#plot continental US
cont_geo_map_data= geo_map_data.drop(geo_map_data[geo_map_data.State == "AK"].index)
base = continent_shape.boundary.plot(figsize=(15, 7), linewidth = 0.3, zorder = 1)
ax = cont_geo_map_data.plot(ax=base, marker='o', column ='Walmart store entry or/and exit (2016, per city)', legend=True, categorical = True, markersize=(cont_geo_map_data["Stores' changes"]*20), cmap = cmap, edgecolors = "black", linewidth = 0.3, zorder=2)
plt.xlabel("Longitude (°)")
plt.ylabel("Latitude (°)")
plt.title("Walmart stores exit and entry per city in the contiguous United States (January, 1, 2016 - January 31, 2017)")
plt.show()

NameError: name 'geo_map_data' is not defined

In [ ]:
#plot state by state
geo_map_data['State_nb']= geo_map_data['county_fips'].astype(str).str[0:-3]
for state in geo_map_data.State.unique():
    fig = plt.figure()
    geo_map_data2= geo_map_data[geo_map_data.State == state]
    statefp = geo_map_data2['State_nb'].iloc[0]
    if len(statefp)==1:
        statefp = '0'+statefp
    base = shape[shape.STATEFP == statefp].boundary.plot(figsize=(15, 7), linewidth = 0.3, zorder = 1)
    ax = geo_map_data2.plot(ax=base, marker='o', column ='Walmart store entry or/and exit (2016, per city)', legend=True, categorical = True, markersize=(cont_geo_map_data["Stores' changes"]*20), cmap = cmap, edgecolors = "black", linewidth = 0.3, zorder=2)
    plt.xlabel("Longitude (°)")
    plt.ylabel("Latitude (°)")
    plt.title(f"{state} - Walmart stores exit and entry per city (January, 1, 2016 - January 31, 2017)")
    plt.show()

For each city where there is at least an entry or exit, the circle's radius is proportional to the number of stores' openings and closings (stores' changes) in the city for year 2016 (from January, 1, 2016 to January 31, 2017).